# Finetuning CTC models on other languages

In previous tutorials, we have seen a few ways to restore an ASR model, set up the data loaders, and then either train from scratch or fine-tune the model on a small dataset. In this tutorial, we extend previous tutorials and discuss in detail how to * fine-tune a pre-trained model onto a new language*. While many of the concepts are similar to previous tutorials, this tutorial will dive deeper into essential steps. Namely,

 - Data preprocessing
 - Prepare tokenizers
 - Discuss how to fine-tune models on low-resource languages efficiently
 - Train a character encoding CTC model
 - Train a sub-word encoding CTC model

For this tutorial (and limited by the compute and storage available on Colab environments), we will attempt to fine-tune an English ASR model onto the [Mozilla Common Voice](https://commonvoice.mozilla.org/en) dataset for Japanese. This dataset will also allow us to discuss a few details for fine-tuning low-resource languages. The methods discussed here can also be applied to languages with several thousand hours of data!

**Note**: It is advised to review the execution flow diagram for ASR models in order to correctly setup the model prior to fine-tuning - [ASR CTC Examples](https://github.com/NVIDIA/NeMo/blob/main/examples/asr/asr_ctc/README.md)


In [ ]:
import os
import copy
from omegaconf import OmegaConf, open_dict
import re

In [6]:
# Nemo import and Nemo utils
import nemo
import nemo.collections.asr as nemo_asr
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager

In [7]:
import prepare_dataset

## Preparing Dataset and direcotries

In [4]:
# Make the directory for scripts and datasets
data_dir = 'datasets/'

if not os.path.exists("scripts"):
  os.makedirs("scripts")

In [6]:
# Tolgo i file invalidati del dataset (Serve solo per i Mozilla Common Voice datasets)
prepare_dataset.delete_invalidated("/workspace/finetuning/datasets/invalidated.tsv")
prepare_dataset.delete_invalidated("/workspace/finetuning/datasets/other.tsv")

Eliminazione file: 100%|██████████| 14549/14549 [16:18<00:00, 14.87file/s]


In [5]:
# Directory dei file audio
clips_dir = os.path.join('datasets', "clips")

In [8]:
# Il dataset scaricato contiene file .mp3, quindi converto a .wav
# Aggiorno anche il path dei file aggiungendo davanti clips_dir
prepare_dataset.convert_mp3_to_wav(clips_dir) # This may took some time... (not multiprocessing or some optimizations)

Converting MP3 to WAV: 100%|██████████| 102579/102579 [4:56:38<00:00,  5.76it/s]  


Conversione completata!


In [7]:
# Prendo il nome dei file che contengono i path degli audio
train_file = os.path.join(data_dir, "train.csv")
validation_file = os.path.join(data_dir, "dev.csv")
test_file = os.path.join(data_dir, "test.csv")

In [10]:
# Converto da tsv a csv e tengo solo le colonne rilevanti (audio_path e testo_audio)
train_file, test_file, validation_file = prepare_dataset.process_tsv(clips_dir, [train_file, test_file, validation_file])

In [8]:
prepare_dataset.remove_mp3(clips_dir)

In [11]:
# Riduco i file da usare perché ho un HW limitato e aggiorno i CSV con le informazioni dei file
reduce_dataset = True
if reduce_dataset:
    prepare_dataset.keep_first_n_lines(train_file, 1000)
    prepare_dataset.keep_first_n_lines(test_file, 100)
    prepare_dataset.keep_first_n_lines(validation_file, 100)

In [ ]:
# Faccio la conversione dei dataset in un formato compatibile con NeMo e genero i manifest contenti audio_path, sentence e duration del file
# Potrebbe settare altre cose di NeMo e fare controlli che sia compatibile (sposta file, controlla integrità ecc)

In [9]:
# Converto i file da MCV to NeMo format
prepare_dataset.MVCtoNEMO(train_file, "train")
prepare_dataset.MVCtoNEMO(validation_file, "validation")
prepare_dataset.MVCtoNEMO(test_file, "test")

Manifest salvato in: datasets/train/train_manifest.json
Manifest salvato in: datasets/validation/validation_manifest.json
Manifest salvato in: datasets/test/test_manifest.json


In [8]:
# Prendo il nome dei file che contengono i path degli audio
data_dir = "datasets"
train_file = os.path.join(data_dir, "train_manifest.json")
validation_file = os.path.join(data_dir, "validation_manifest.json")
test_file = os.path.join(data_dir, "test_manifest.json")

In [9]:
prepare_dataset.normalize_text(train_file)
prepare_dataset.normalize_text(validation_file)
prepare_dataset.normalize_text(test_file)

File JSON normalizzato e salvato in: datasets/train_manifest.json
File JSON normalizzato e salvato in: datasets/validation_manifest.json
File JSON normalizzato e salvato in: datasets/test_manifest.json


In [ ]:
import pandas as pd
def reorder_and_rename_columns(json_file):
    # Legge il file JSON in un DataFrame
    df = pd.read_json(json_file, lines=True)

    # Rinomina la colonna "audio" in "audio_filepath"
    df = df.rename(columns={"audio": "audio_filepath"})
    
    # Riordina le colonne
    ordered_df = df[['text', 'audio_filepath', 'duration']]

    # Salva il DataFrame con le colonne rinominate e riordinate nel file JSON
    ordered_df.to_json(json_file, orient="records", lines=True, force_ascii=False)
    print(f"File JSON con colonne rinominate e riordinate salvato in: {json_file}")

In [14]:
reorder_and_rename_columns(train_file)
reorder_and_rename_columns(test_file)
reorder_and_rename_columns(validation_file)

File JSON con colonne rinominate e riordinate salvato in: datasets/train_manifest.json
File JSON con colonne rinominate e riordinate salvato in: datasets/test_manifest.json
File JSON con colonne rinominate e riordinate salvato in: datasets/validation_manifest.json


I Downloaded from https://commonvoice.mozilla.org/it/datasets the dataset 'Common Voice Corpus 3' of 1 GB. In realtà è solo una piccola parte di un dataset. MVC rilascia dataset interi e piccoli aggiornamenti chiamati Delta Segment, io ho installato uno di questi. 

In [10]:
LANGUAGE = "it"
tokenizer_dir = os.path.join('tokenizers', LANGUAGE) 
manifest_dir = data_dir

Now that the dataset has been downloaded, let's prepare some paths to easily access the manifest files for the train, dev, and test partitions.

# Preparing the manifest

Before we start training the model on the above unprocessed manifest files, we need to analyze the data. Data pre-processing is perhaps the most essential task, and often requires moderate expertise in the language.

While we could technically use the manifests above to train a model, the results would potentially be abysmal. Let's dive a little deeper into what challenges this dataset poses to our models.

**Note**: The pre-processing done on this corpus is specifically done to reduce ambiguity in transcripts, due to the minuscule amount of data we possess. Given enough data, the models discussed here could potentially learn well, even without such heavy pre-processing.

In [ ]:
# Salvo il path dei manifest creati con lo script NVIDIA
train_manifest = f"{manifest_dir}/train/train/train_manifest.json"
dev_manifest = f"{manifest_dir}/validation/validation/validation_manifest.json"
test_manifest = f"{manifest_dir}/test/test/test_manifest.json"

### Manifest utilities

First, we construct some utilities to read and write manifest files

In [ ]:
# Manifest Utils
from tqdm.auto import tqdm
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest, write_manifest
import json

def write_processed_manifest(data, original_path):
    original_manifest_name = os.path.basename(original_path)
    new_manifest_name = original_manifest_name.replace(".json", "_processed.json")

    manifest_dir = os.path.split(original_path)[0]
    filepath = os.path.join(manifest_dir, new_manifest_name)
    write_manifest(filepath, data)
    print(f"Finished writing manifest: {filepath}")
    return filepath

In [ ]:
# Leggo i manifest e salvo il contenuto
train_manifest_data = read_manifest(train_manifest)
dev_manifest_data = read_manifest(dev_manifest)
test_manifest_data = read_manifest(test_manifest)

Next, we extract just the text corpus from the manifest.

In [ ]:
# Estraggo il testo dal manifest per poterne valutare i caratteri
train_text = [data['text'] for data in train_manifest_data]
dev_text = [data['text'] for data in dev_manifest_data]
test_text = [data['text'] for data in test_manifest_data]

In [ ]:
# Tengo solo poche righe per limiti HW
if reduce_dataset:
    prepare_dataset.keep_first_n_lines(train_manifest, 1000)
    prepare_dataset.keep_first_n_lines(test_manifest, 100)
    prepare_dataset.keep_first_n_lines(dev_manifest, 100)

-----

## Character Processing

Let us calculate the character set - which is the set of unique tokens that exist within the text manifests.

In [ ]:
from collections import defaultdict
import string

# Ritorna i caratteri contenuti nel manifest manifest_data
def get_charset(manifest_data):
    charset = defaultdict(int)
    for row in tqdm(manifest_data, desc="Computing character set"):
        text = row['text']
        for character in text:
            charset[character] += 1
    return charset

# Stampa le lettere mancanti dal set dato in input
def lettere_mancanti(dizionario):
    alfabeto = set(string.ascii_letters)
    lettere_presenti = set(dizionario.keys())
    lettere_mancanti = alfabeto - lettere_presenti
    return sorted(lettere_mancanti)

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\…\{\}\(\)\[\]\–\’\'\/]'  # Caratteri da ignorare

# Rimuove i caratteri di chars_to_ignore_regex
def remove_special_characters(data):
    data["text"] = re.sub(chars_to_ignore_regex, '', data["text"]).lower().strip()
    return data

# Applica i filtri sui caratteri 
def apply_preprocessors(manifest, preprocessors):
    for processor in preprocessors:
        for idx in tqdm(range(len(manifest)), desc=f"Applying {processor.__name__}"):
            manifest[idx] = processor(manifest[idx])

    print("Finished processing manifest !")
    return manifest

In [ ]:
# Ottengo i set di caratteri di ogni set
train_charset = get_charset(train_manifest_data)
dev_charset = get_charset(dev_manifest_data)
test_charset = get_charset(test_manifest_data)

# Stampo le lettere mancanti (dell'alfabeto) di ogni set (differenza tra maiuscole e minuscole come con ASCII)
print("Lettere Mancanti da: train, validation e test")
print(lettere_mancanti(train_charset))
print(lettere_mancanti(dev_charset))
print(lettere_mancanti(test_charset))
print()

# Conto il numero di caratteri unici nei tre split
train_dev_set = set.union(set(train_charset.keys()), set(dev_charset.keys()))
test_set = set(test_charset.keys())
print(f"Numero di caratteri nel train+dev set : {len(train_dev_set)}")
print(f"Numero di caratteri nel test set : {len(test_set)}")
print()

# List of pre-processing functions
PREPROCESSORS = [
    remove_special_characters
    # other functions for complex languages
]

# Load manifests
print("Leggo i manifest...")
train_data = read_manifest(train_manifest)
dev_data = read_manifest(dev_manifest)
test_data = read_manifest(test_manifest)
print()

# Applico i filtri inseriti in PREPROCESSORS
print("Applico i filtri...")
train_data_processed = apply_preprocessors(train_data, PREPROCESSORS)
dev_data_processed = apply_preprocessors(dev_data, PREPROCESSORS)
test_data_processed = apply_preprocessors(test_data, PREPROCESSORS)
print()

# Aggiorno i manifest con i filtri applicati
print("Aggiorno i manifest...")
train_manifest_cleaned = write_processed_manifest(train_data_processed, train_manifest)
dev_manifest_cleaned = write_processed_manifest(dev_data_processed, dev_manifest)
test_manifest_cleaned = write_processed_manifest(test_data_processed, test_manifest)
print()

# Aggiorno i set di caratteri modificati 
print("Rileggo i caratteri aggionati...")
train_manifest_data = read_manifest(train_manifest_cleaned)
train_charset = get_charset(train_manifest_data)

dev_manifest_data = read_manifest(dev_manifest_cleaned)
dev_charset = get_charset(dev_manifest_data)

test_manifest_data = read_manifest(test_manifest_cleaned)
test_charset = get_charset(test_manifest_data)
print()

# Aggiorno il nuovo set di caratteri finale di test e validation
train_dev_set = sorted(set.union(set(train_charset.keys()), set(dev_charset.keys())))
test_set = set(train_charset.keys())

vocaboulary = " ".join(train_dev_set)
print("Nuovo character set: " + vocaboulary)

# Character Encoding CTC Model

Now that we have a processed dataset, we can begin training an ASR model on this dataset. The following section will detail how we prepare a CTC model which utilizes a Character Encoding scheme.

This section will utilize a pre-trained [QuartzNet 15x5](https://arxiv.org/abs/1910.10261), which has been trained on roughly 7,000 hours of English speech base model. We will modify the decoder layer (thereby changing the model's vocabulary) and then train for a small number of epochs.

In [ ]:
# Scarico il modello inglese per cui devo fare fine-tuning
char_model = nemo_asr.models.ASRModel.from_pretrained("stt_en_quartznet15x5", map_location='cpu')

### Update the vocabulary

Changing the vocabulary of a character encoding ASR model is as simple as passing the list of new tokens that comprise the vocabulary as input to `change_vocabulary()`.

In [ ]:
# Cambio il vocabolario delle lettere
#char_model.change_vocabulary(new_vocabulary=list(train_dev_set))

### Freezing the encoder

In [ ]:
# Blocco l'encoder del Transformer
freeze_encoder = True

In [ ]:
import torch
import torch.nn as nn

def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

if freeze_encoder:
  char_model.encoder.freeze()
  char_model.encoder.apply(enable_bn_se)
  logging.info("Model encoder has been frozen, and batch normalization has been unfrozen")
else:
  char_model.encoder.unfreeze()
  logging.info("Model encoder has been un-frozen")

### Update config

Each NeMo model has a config embedded in it, which can be accessed via `model.cfg`. In general, this is the config that was used to construct the model.

For pre-trained models, this config generally represents the config used to construct the model when it was trained. A nice benefit to this embedded config is that we can repurpose it to set up new data loaders, optimizers, schedulers, and even data augmentation!

In [ ]:
# Aggiorno le label del modello
char_model.cfg.labels = list(train_dev_set)

In [ ]:
# Creo una copia delle configuazioni del modello per poterle modificare
cfg = copy.deepcopy(char_model.cfg)

### Setting up data loaders

Now that the model's character set has been updated let's prepare the model to utilize the new character set even in the data loaders. Note that this is crucial so that the data produced during training/validation matches the new character set, and tokens are encoded/decoded correctly.

**Note**: An important config parameter is `normalize_transcripts` and `parser`. There are some parsers that are used for specific languages for character based models - currently only `en` is supported. These parsers will preprocess the text with the given languages parser. However, for other languages, it is advised to explicitly set `normalize_transcripts = False` - which will prevent the parser from processing text.

In [ ]:
import json
from omegaconf import OmegaConf

# Converte DictConfig in un dizionario Python
cfg_dict = OmegaConf.to_container(cfg, resolve=True)

# Stampa in formato JSON con indentazione
print(json.dumps(cfg_dict, indent=4))

In [ ]:
# Setup train, validation, test configs
with open_dict(cfg):
  # Train dataset  (Concatenate train manifest cleaned and dev manifest cleaned)
  cfg.train_ds.manifest_filepath = f"{train_manifest_cleaned},{dev_manifest_cleaned}"
  cfg.train_ds.labels = list(train_dev_set)
  cfg.train_ds.normalize_transcripts = False
  cfg.train_ds.batch_size = 8#32
  cfg.train_ds.num_workers = 0#4#8
  cfg.train_ds.pin_memory = True
  cfg.train_ds.trim_silence = True

  # Validation dataset  (Use test dataset as validation, since we train using train + dev)
  cfg.validation_ds.manifest_filepath = test_manifest_cleaned
  cfg.validation_ds.labels = list(train_dev_set)
  cfg.validation_ds.normalize_transcripts = False
  cfg.validation_ds.batch_size = 4#8
  cfg.validation_ds.num_workers = 0#4#8
  cfg.validation_ds.pin_memory = True
  cfg.validation_ds.trim_silence = True

In [ ]:
# setup data loaders with new configs
char_model.setup_training_data(cfg.train_ds)
char_model.setup_multiple_validation_data(cfg.validation_ds)
# Mi dice 2000 (train + validation) e 1000 di test

### Setting up optimizer and scheduler

When fine-tuning character models, it is generally advised to use a lower learning rate and reduced warmup. A reduced learning rate helps preserve the pre-trained weights of the encoder. Since the fine-tuning dataset is generally smaller than the original training dataset, the warmup steps would be far too much for the smaller fine-tuning dataset.

-----
**Note**: When freezing the encoder, it is possible to use the original learning rate as the model was trained on. The original learning rate can be used because the encoder is frozen, so the learning rate is used only to optimize the decoder. However, a very high learning rate would still destabilize training, even with a frozen encoder.

In [ ]:
# Original optimizer + scheduler
print(OmegaConf.to_yaml(char_model.cfg.optim))

In [ ]:
#with open_dict(char_model.cfg.optim):
#  char_model.cfg.optim.lr = 0.01
#  char_model.cfg.optim.betas = [0.95, 0.5]  # from paper
#  char_model.cfg.optim.weight_decay = 0.001  # Original weight decay
#  char_model.cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
#  char_model.cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
#  char_model.cfg.optim.sched.min_lr = 1e-3#1e-5
with open_dict(cfg.optim):
  cfg.optim.lr = 0.01
  cfg.optim.betas = [0.95, 0.5]  # from paper
  cfg.optim.weight_decay = 0.001  # Original weight decay
  cfg.optim.sched.warmup_steps = None  # Remove default number of steps of warmup
  cfg.optim.sched.warmup_ratio = 0.05  # 5 % warmup
  cfg.optim.sched.min_lr = 1e-5
  # ho trovato anche una  versione con 1e-3 ma era con un altro modello e altri settings (usava tokenizer)

### Setting up augmentation

Remember that the model was trained on several thousands of hours of data, so the regularization provided to it might not suit the current dataset. We can easily change it as we see fit.

-----

You might notice that we utilize `char_model.from_config_dict()` to create a new SpectrogramAugmentation object and assign it directly in place of the previous augmentation. This is generally the syntax to be followed whenever you notice a `_target_` tag in the config of a model's inner config.

-----
**Note**: For low resource languages, it might be better to increase augmentation via SpecAugment to reduce overfitting. However, this might, in turn, make it too hard for the model to train in a short number of epochs.

In [ ]:
print(OmegaConf.to_yaml(char_model.cfg.spec_augment))

### Augmentation

Note: For low resource languages, it might be better to increase augmentation via SpecAugment to reduce overfitting. However, this might, in turn, make it too hard for the model to train in a short number of epochs.

In [ ]:
# Ha senso scommentare solo se alziamo il nuemero di EPOCH
#with open_dict(char_model.cfg.spec_augment):
#    char_model.cfg.spec_augment.freq_masks = 2      # Numero di maschere di frequenza
#    char_model.cfg.spec_augment.freq_width = 25     # Ampiezza massima delle bande di frequenza mascherate
#    char_model.cfg.spec_augment.time_masks = 2      # Numero di maschere temporali
#    char_model.cfg.spec_augment.time_width = 0.05   # Percentuale del tempo totale mascherato

char_model.spec_augmentation = char_model.from_config_dict(char_model.cfg.spec_augment)

### Setup Metrics

Originally, the model was trained on an English dataset corpus. When calculating Word Error Rate, we can easily use the "space" token as a separator for word boundaries. On the other hand, certain languages such as Japanese and Mandarin do not use "space" tokens, instead opting for different ways to annotate the end of the word.

In cases where the "space" token is not used to denote a word boundary, we can use the Character Error Rate metric instead, which computes the edit distance at a token level rather than a word level.

We might also be interested in noting model predictions during training and inference. As such, we can enable logging of the predictions.

In [ ]:
use_cer = False # tiene gli spazi bianchi come separatori tra parole
log_prediction = True
char_model.wer.use_cer = use_cer
char_model.wer.log_prediction = log_prediction

## Setup Trainer and Experiment Manager

And that's it! Now we can train the model by simply using the Pytorch Lightning Trainer and NeMo Experiment Manager as always.

For demonstration purposes, the number of epochs is kept intentionally low. Reasonable results can be obtained in around 100 epochs (approximately 25 minutes on Colab GPUs).

In [ ]:
import torch
import pytorch_lightning as ptl

if torch.cuda.is_available():
  accelerator = 'gpu'
else:
  accelerator = 'cpu'

EPOCHS = 5

trainer = ptl.Trainer(devices=1,
                      accelerator=accelerator,
                      max_epochs=EPOCHS,
                      accumulate_grad_batches=1,
                      enable_checkpointing=False,
                      logger=True,
                      log_every_n_steps=5,
                      check_val_every_n_epoch=10)

# Setup model with the trainer
char_model.set_trainer(trainer)

# Finally, update the model's internal config
char_model.cfg = cfg

In [ ]:
save_path = f"Model-{LANGUAGE}.nemo"
char_model.save_to(f"{save_path}")
print(f"Model saved at path : {os.getcwd() + os.path.sep + save_path}")